In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 51.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=7)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-10 18:26:08,017] A new study created in memory with name: no-name-248bd2c7-7a7c-44e6-a5f4-8809beb3f3c5
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.272128,0.902747,0.685198,0.475694,0.528919
2,No log,0.217040,0.924176,0.807798,0.614583,0.689117
3,No log,0.209377,0.921429,0.761003,0.642361,0.694252


***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in label

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-10 18:27:01,551] Trial 0 finished with value: 0.6942521488333181 and parameters: {'learning_rate': 2.556527112363619e-05, 'batch_size': 4, 'num_train_epochs': 3, 'weight_decay': 0.05891980498042266}. Best is trial 0 with value: 0.6942521488333181.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.203890,0.924725,0.766572,0.666667,0.711705
1,No log,0.193068,0.929670,0.777925,0.694444,0.729797
2,No log,0.200638,0.924176,0.751497,0.687500,0.716328


...

[I 2024-12-10 18:37:29,988] Trial 8 finished with value: 0.7470616699629434 and parameters: {'learning_rate': 3.617974147894206e-05, 'batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.0028619470369510157}. Best is trial 6 with value: 0.7702629188318819.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: T

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.280814,0.920330,0.753586,0.736111,0.738147
2,No log,0.280713,0.921978,0.754516,0.736111,0.736990
3,No log,0.278693,0.921429,0.754938,0.732639,0.736970
4,No log,0.279272,0.921429,0.757485,0.729167,0.734941
5,No log,0.278609,0.923077,0.761804,0.736111,0.742012
6,No log,0.278369,0.921978,0.757759,0.732639,0.738471
7,No log,0.278871,0.922527,0.759475,0.732639,0.739484


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16


[I 2024-12-10 18:39:06,600] Trial 9 finished with value: 0.7394840927884965 and parameters: {'learning_rate': 2.634793934318445e-06, 'batch_size': 8, 'num_train_epochs': 7, 'weight_decay': 0.048883603138419905}. Best is trial 6 with value: 0.7702629188318819.


Best trial:
FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.7702629188318819], datetime_start=datetime.datetime(2024, 12, 10, 18, 33, 56, 72562), datetime_complete=datetime.datetime(2024, 12, 10, 18, 35, 24, 130304), params={'learning_rate': 1.1603919360800287e-05, 'batch_size': 16, 'num_train_epochs': 7, 'weight_decay': 0.006845868059871536}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=6, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"<<==============Training Time==================>>")
print(f"Training time: {training_time:.2f} seconds")
print(f"<<=============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 224


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.286014,0.928022,0.776905,0.760417,0.761294
1,No log,0.302354,0.920879,0.755038,0.729167,0.729056
2,No log,0.282769,0.924176,0.765442,0.750000,0.752844
3,No log,0.293419,0.924176,0.764433,0.739583,0.741392
4,No log,0.285475,0.925275,0.763382,0.746528,0.747947
5,No log,0.289136,0.924725,0.764038,0.743056,0.745249
6,No log,0.290731,0.924176,0.762618,0.739583,0.742249


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




<<==============Training Time==================>>
Training time: 86.69 seconds
<<=============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


{'eval_loss': 0.2907311022281647, 'eval_accuracy': 0.9241758241758242, 'eval_precision': 0.7626180083006228, 'eval_recall': 0.7395833333333334, 'eval_f1': 0.7422490082709581, 'eval_runtime': 0.9494, 'eval_samples_per_second': 273.853, 'eval_steps_per_second': 9.48, 'epoch': 6.98}


### Model Saving

In [14]:
!mkdir 'best_model_scibert_scivocab_uncased_pharo'
!mkdir 'best_model_scibert_scivocab_uncased_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_scibert_scivocab_uncased_pharo')
tokenizer.save_pretrained('./best_model_scibert_scivocab_uncased_pharo_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_scibert_scivocab_uncased_pharo')
tokenizer = AutoTokenizer.from_pretrained('./best_model_scibert_scivocab_uncased_pharo_tokenizer')

Saving model checkpoint to ./best_model_scibert_scivocab_uncased_pharo
Configuration saved in ./best_model_scibert_scivocab_uncased_pharo/config.json
Model weights saved in ./best_model_scibert_scivocab_uncased_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_scibert_scivocab_uncased_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_scibert_scivocab_uncased_pharo/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_scibert_scivocab_uncased_pharo",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initia

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 289
  Batch size = 32


Evaluation Metrics: {'accuracy': 0.9218981710331191, 'precision': 0.7499572630386584, 'recall': 0.717607973421927, 'f1': 0.7288221882197249}
Inference Time: 1.14 seconds


In [18]:
!zip -r best_model_scibert_scivocab_uncased_pharo.zip './best_model_scibert_scivocab_uncased_pharo'
!zip -r best_model_scibert_scivocab_uncased_pharo_tokenizer.zip './best_model_scibert_scivocab_uncased_pharo_tokenizer'

  adding: best_model_scibert_scivocab_uncased_pharo/ (stored 0%)
  adding: best_model_scibert_scivocab_uncased_pharo/pytorch_model.bin (deflated 7%)
  adding: best_model_scibert_scivocab_uncased_pharo/config.json (deflated 54%)
  adding: best_model_scibert_scivocab_uncased_pharo/training_args.bin (deflated 51%)
  adding: best_model_scibert_scivocab_uncased_pharo_tokenizer/ (stored 0%)
  adding: best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer.json (deflated 71%)
  adding: best_model_scibert_scivocab_uncased_pharo_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_scibert_scivocab_uncased_pharo_tokenizer/vocab.txt (deflated 52%)


In [20]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_scibert_scivocab_uncased_pharo.zip "/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo/"
!cp best_model_scibert_scivocab_uncased_pharo_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo/best_model_scibert_scivocab_uncased_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo_Tokenizer/best_model_scibert_scivocab_uncased_pharo_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo/best_model_scibert_scivocab_uncased_pharo.zip
replace ./best_model_scibert_scivocab_uncased_pharo/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_scibert_scivocab_uncased_pharo/pytorch_model.bin  
  inflating: ./best_model_scibert_scivocab_uncased_pharo/config.json  
  inflating: ./best_model_scibert_scivocab_uncased_pharo/training_args.bin  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_pharo_Tokenizer/best_model_scibert_scivocab_uncased_pharo_tokenizer.zip
replace ./best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer.json  
  inflating: ./best_model_scibert_scivocab_uncased_pharo_tokenizer/special_tokens_map.json  
  inflating: ./best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer_config.json  


In [23]:
best_model_scibert_scivocab_uncased_pharo = 'best_model_scibert_scivocab_uncased_pharo'
best_model_scibert_scivocab_uncased_pharo_tokenizer = 'best_model_scibert_scivocab_uncased_pharo_tokenizer'

In [24]:
model = AutoModelForSequenceClassification.from_pretrained(best_model_scibert_scivocab_uncased_pharo)
tokenizer = AutoTokenizer.from_pretrained(best_model_scibert_scivocab_uncased_pharo_tokenizer)

loading configuration file best_model_scibert_scivocab_uncased_pharo/config.json
Model config BertConfig {
  "_name_or_path": "best_model_scibert_scivocab_uncased_pharo",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "

In [25]:
text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.2f} seconds")
output

Inference Time: 0.12 seconds


SequenceClassifierOutput(loss=None, logits=tensor([[-6.8025, -5.1443,  4.4330, -7.2159, -6.9669, -6.0245, -7.5425]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3
